In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

from utils.dataset_tools import DatasetTools

In [2]:
DATASET_PATH = '../dataset/larger_full_datasets.csv'
df = pd.read_csv(DATASET_PATH)
df.head()


,sample_id,frame,label,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,landmark_3_x,...,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z,landmark_21_x,landmark_21_y,landmark_21_z
0,1,0,A,0.345417,0.648431,-1.125724e-07,0.363545,0.620765,-0.010274,0.367171,...,-0.003865,0.321722,0.551512,-0.008457,0.326903,0.565844,-0.005724,0.329682,0.577548,-0.001789
1,1,1,A,0.366266,0.597102,-9.622672e-08,0.391580,0.578109,-0.012907,0.406190,...,0.000685,0.356418,0.497935,-0.004451,0.361665,0.515749,-0.001988,0.363353,0.523833,0.002337
2,1,2,A,0.400966,0.572690,-1.089629e-07,0.424555,0.552354,-0.004017,0.439723,...,-0.004296,0.383425,0.481909,-0.009304,0.389207,0.502081,-0.005430,0.391897,0.513775,-0.000924
3,1,3,A,0.418257,0.573744,-1.190043e-07,0.441344,0.563031,-0.007786,0.460869,...,-0.002903,0.402536,0.489501,-0.009669,0.406139,0.510639,-0.006567,0.409394,0.523706,-0.001918
4,1,4,A,0.431913,0.582072,-1.241054e-07,0.453692,0.577044,-0.009167,0.474907,...,-0.004793,0.417201,0.501486,-0.012369,0.419653,0.524298,-0.008715,0.422120,0.535854,-0.003439


In [3]:
sequences, labels = DatasetTools.divide_into_sequences(df)

In [4]:
sequences = np.array([DatasetTools.normalize_wrist(seq) for seq in sequences])

print("Sekwencje:", sequences.shape)
print("Etykiety:", labels.shape)


Sekwencje: (24609, 30, 63)
Etykiety: (24609,)


In [5]:
aug_config = {
    'flip': True,
    'gaussian_noise': True,
    'speckle_noise': True,
    'salt_pepper': True,
}

batch_size = 32
generator = DatasetTools.augment_generator(sequences, labels, batch_size, aug_config)

all_sequences, all_labels = [], []

for X_batch, y_batch in generator:
    all_sequences.append(X_batch)
    all_labels.append(y_batch)

all_sequences = np.concatenate(all_sequences, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

all_sequences = all_sequences.astype(np.float32)

print("Po augmentacji:", all_sequences.shape, all_labels.shape)


I0000 00:00:1758557993.201891   79300 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1525 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1


Po augmentacji: (24609, 30, 63) (24609,)


In [12]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(all_labels)  # A...Z -> 0...N
print("Przykładowe etykiety zakodowane:", y_encoded[:10])

Przykładowe etykiety zakodowane: [0 0 0 0 0 0 0 0 0 0]


In [11]:
flat_sequences = all_sequences.reshape(all_sequences.shape[0], -1)

In [8]:
import pandas as pd

def save_sequences_to_csv(sequences, labels, save_path):
    """
    Zapisuje sekwencje w formacie datasetu:
    sample_id, frame, label, landmark_1_x ... landmark_21_z
    """
    num_sequences, num_frames, num_features = all_sequences.shape
    sample_ids = np.repeat(np.arange(num_sequences), num_frames)
    frame_ids = np.tile(np.arange(num_frames), num_sequences)
    labels_col = np.repeat(all_labels, num_frames)

    flat_data = all_sequences.reshape(num_sequences*num_frames, num_features)
    df_out = pd.DataFrame(flat_data, columns=[f"landmark_{i+1}_{axis}"
                                              for i in range(21) for axis in ['x','y','z']])
    df_out.insert(0, "label", labels_col)
    df_out.insert(0, "frame", frame_ids)
    df_out.insert(0, "sample_id", sample_ids)

    df_out.to_csv(save_path, index=False, float_format="%.6f")


    print(f"Dane zapisano do pliku: {save_path}")


# przykład użycia
save_sequences_to_csv(all_sequences, all_labels, "../dataset/augmented_dataset.csv")


Dane zapisano do pliku: ../dataset/augmented_dataset.csv


In [13]:
# Przed augmentacją
print("PRZED augmentacją:")
print("Liczba gestów (sekwencji):", len(sequences))

unique, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"{label}: {count} gestów")

# Po augmentacji
print("\nPO augmentacji:")
print("Liczba gestów (sekwencji):", len(all_sequences))

unique_aug, counts_aug = np.unique(all_labels, return_counts=True)
for label, count in zip(unique_aug, counts_aug):
    print(f"{label}: {count} gestów")


PRZED augmentacją:
Liczba gestów (sekwencji): 24609
A: 714 gestów
A_A: 681 gestów
B: 682 gestów
C: 682 gestów
CH: 680 gestów
CZ: 681 gestów
C_C: 712 gestów
D: 682 gestów
E: 673 gestów
E_E: 682 gestów
F: 682 gestów
G: 682 gestów
H: 682 gestów
I: 682 gestów
J: 682 gestów
K: 682 gestów
L: 682 gestów
L_L: 682 gestów
M: 682 gestów
N: 681 gestów
N_N: 682 gestów
O: 682 gestów
O_O: 682 gestów
P: 682 gestów
R: 683 gestów
RZ: 681 gestów
S: 682 gestów
SZ: 680 gestów
S_S: 682 gestów
T: 682 gestów
U: 682 gestów
W: 682 gestów
Y: 682 gestów
Z: 691 gestów
Z_X: 683 gestów
Z_Z: 683 gestów

PO augmentacji:
Liczba gestów (sekwencji): 24609
A: 714 gestów
A_A: 681 gestów
B: 682 gestów
C: 682 gestów
CH: 680 gestów
CZ: 681 gestów
C_C: 712 gestów
D: 682 gestów
E: 673 gestów
E_E: 682 gestów
F: 682 gestów
G: 682 gestów
H: 682 gestów
I: 682 gestów
J: 682 gestów
K: 682 gestów
L: 682 gestów
L_L: 682 gestów
M: 682 gestów
N: 681 gestów
N_N: 682 gestów
O: 682 gestów
O_O: 682 gestów
P: 682 gestów
R: 683 gestów
RZ: 681 

In [3]:
# Assuming you have the corrected DatasetTools class defined above

# Rest of your script
DATASET_PATH = '../dataset/larger_full_datasets.csv'
df = pd.read_csv(DATASET_PATH)
# df.head() # Already in your original code

sequences, labels = DatasetTools.divide_into_sequences(df)
sequences = np.array([DatasetTools.normalize_wrist(seq) for seq in sequences])

print("Sekwencje (przed augmentacją):", sequences.shape)
print("Etykiety (przed augmentacją):", labels.shape)
aug_config = {
    'flip': True,
    'gaussian_noise': True,
    'gaussian_std': 0.02,
    'speckle_noise': True,
    'speckle_std': 0.01,
    'salt_pepper': True,
    'salt_pepper_prob': 0.02,
}

multiplier = 3     # wygeneruje do 3 przykładów na oryginał (wliczając oryginał jeśli include_original=True)
include_original = True

batch_size = 32
generator = DatasetTools.augment_generator(sequences, labels, batch_size,
                                           aug_config,
                                           multiplier=multiplier,
                                           include_original=include_original,
                                           random_state=42)

all_augmented_sequences, all_augmented_labels = [], []
for X_batch, y_batch in generator:
    all_augmented_sequences.append(X_batch)
    all_augmented_labels.append(y_batch)

all_augmented_sequences = np.concatenate(all_augmented_sequences, axis=0)
all_augmented_labels = np.concatenate(all_augmented_labels, axis=0)

all_augmented_sequences = all_augmented_sequences.astype(np.float32)

print("Po augmentacji (wszystkie zebrane):", all_augmented_sequences.shape, all_augmented_labels.shape)

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(all_augmented_labels)
print("Przykładowe etykiety zakodowane (po augmentacji):", y_encoded[:10])

# This flatten step might not be needed if your model expects 3D sequences
# flat_sequences = all_augmented_sequences.reshape(all_augmented_sequences.shape[0], -1)

# Function to save augmented data (copied from your original)
def save_sequences_to_csv(sequences_to_save, labels_to_save, save_path):
    num_sequences, num_frames, num_features = sequences_to_save.shape
    sample_ids = np.repeat(np.arange(num_sequences), num_frames)
    frame_ids = np.tile(np.arange(num_frames), num_sequences)
    labels_col = np.repeat(labels_to_save, num_frames)

    flat_data = sequences_to_save.reshape(num_sequences*num_frames, num_features)
    df_out = pd.DataFrame(flat_data, columns=[f"landmark_{i+1}_{axis}"
                                              for i in range(21) for axis in ['x','y','z']])
    df_out.insert(0, "label", labels_col)
    df_out.insert(0, "frame", frame_ids)
    df_out.insert(0, "sample_id", sample_ids)

    df_out.to_csv(save_path, index=False, float_format="%.6f")
    print(f"Dane zapisano do pliku: {save_path}")

# Example usage with augmented data
save_sequences_to_csv(all_augmented_sequences, all_augmented_labels, "../dataset/augmented_dataset.csv")

# Statistics before and after
print("PRZED augmentacją:")
print("Liczba gestów (sekwencji):", len(sequences))
unique, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"{label}: {count} gestów")

print("\nPO augmentacji:")
print("Liczba gestów (sekwencji):", len(all_augmented_sequences))
unique_aug, counts_aug = np.unique(all_augmented_labels, return_counts=True)
for label, count in zip(unique_aug, counts_aug):
    print(f"{label}: {count} gestów")

Sekwencje (przed augmentacją): (24609, 30, 63)
Etykiety (przed augmentacją): (24609,)
Po augmentacji (wszystkie zebrane): (73827, 30, 63) (73827,)
Przykładowe etykiety zakodowane (po augmentacji): [16 16 15 27 17 26  6 33 27  0]
Dane zapisano do pliku: ../dataset/augmented_dataset.csv
PRZED augmentacją:
Liczba gestów (sekwencji): 24609
A: 714 gestów
A_A: 681 gestów
B: 682 gestów
C: 682 gestów
CH: 680 gestów
CZ: 681 gestów
C_C: 712 gestów
D: 682 gestów
E: 673 gestów
E_E: 682 gestów
F: 682 gestów
G: 682 gestów
H: 682 gestów
I: 682 gestów
J: 682 gestów
K: 682 gestów
L: 682 gestów
L_L: 682 gestów
M: 682 gestów
N: 681 gestów
N_N: 682 gestów
O: 682 gestów
O_O: 682 gestów
P: 682 gestów
R: 683 gestów
RZ: 681 gestów
S: 682 gestów
SZ: 680 gestów
S_S: 682 gestów
T: 682 gestów
U: 682 gestów
W: 682 gestów
Y: 682 gestów
Z: 691 gestów
Z_X: 683 gestów
Z_Z: 683 gestów

PO augmentacji:
Liczba gestów (sekwencji): 73827
A: 2142 gestów
A_A: 2043 gestów
B: 2046 gestów
C: 2046 gestów
CH: 2040 gestów
CZ: 2043